In [15]:
import cPickle as pickle
import numpy as np
f = open('/home/mohammad/img_label_rgb.p','rb')
label=pickle.load(f)
f.close()

f = open('/home/mohammad/img_data_rgb.p','rb')
data=pickle.load(f)
f.close()

In [16]:
label.shape, data.shape

((22424,), (22424, 3, 50, 50))

In [17]:
from sklearn.model_selection import train_test_split
X_train1, X_test, y_train1, y_test = train_test_split(data, label, test_size=0.33, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train1, y_train1, test_size=0.2, random_state=42)
print X_train.shape,X_valid.shape, X_test.shape,y_test.shape,y_train.shape

(12019, 3, 50, 50) (3005, 3, 50, 50) (7400, 3, 50, 50) (7400,) (12019,)


In [18]:
import tensorflow as tf
import sys

In [19]:
sys.path.append('/home/mohammad/anaconda/lib/python2.7/site-packages')
import keras
from keras.utils import np_utils

In [21]:
image_size=50
num_labels=10
def reformat(dataset1):
    dataset1 = dataset1.reshape(dataset1.shape[0], image_size, image_size,3).astype(np.float32)
    #dataset2 = dataset2.reshape(dataset2.shape[0], 1, image_size, image_size)
    #dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    #labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    #labels = labels.reshape((-1,10))
    return dataset1
X_train = reformat(X_train)
X_valid = reformat(X_valid)
X_test = reformat(X_test)
print('Training set', X_train.shape)
print('Validation set', X_valid.shape)
#print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (12019, 50, 50, 3))
('Validation set', (3005, 50, 50, 3))


In [22]:
y_train = np_utils.to_categorical(y_train, num_labels)
y_test = np_utils.to_categorical(y_test, num_labels)
y_valid = np_utils.to_categorical(y_valid, num_labels)
print('Training set', y_train.shape)
print('Validation set',y_valid.shape)

('Training set', (12019, 10))
('Validation set', (3005, 10))


In [23]:
# input image dimensions
img_rows, img_cols = 50,50
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)
nb_classes = 10
input_shape = (img_rows, img_cols,1) ##this is tf backend, for theano (1,50,50)

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.constraints import maxnorm
#from keras import backend as K


model = Sequential()

model = Sequential()
model.add(Convolution2D(48, 3, 3, input_shape=(50, 50,3), activation='relu', border_mode='same'))
model.add(Dropout(0.2))
model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same'))
model.add(Dropout(0.2))
model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same'))
model.add(Dropout(0.2))
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))



model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'],
              validation_data=(X_valid, y_valid))

In [28]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_25 (Convolution2D) (None, 50, 50, 48)    1344        convolution2d_input_6[0][0]      
____________________________________________________________________________________________________
dropout_21 (Dropout)             (None, 50, 50, 48)    0           convolution2d_25[0][0]           
____________________________________________________________________________________________________
convolution2d_26 (Convolution2D) (None, 50, 50, 32)    13856       dropout_21[0][0]                 
____________________________________________________________________________________________________
maxpooling2d_13 (MaxPooling2D)   (None, 25, 25, 32)    0           convolution2d_26[0][0]           
___________________________________________________________________________________________

In [29]:
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train, nb_epoch=2, batch_size=32,verbose=1)

/home/mohammad/anaconda/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:1113: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Epoch 1/2
12019/12019 [==============================] - 322s - loss: 2.3013 - acc: 0.1098   
Epoch 2/2
12019/12019 [==============================] - 320s - loss: 2.3001 - acc: 0.1114   


In [24]:
from sklearn.metrics import log_loss
predictions_valid = model.predict(X_valid, batch_size=128, verbose=1)
score = log_loss(y_valid, predictions_valid)
print('Score log_loss: ', score)

3005/3005 [==============================] - 0s     
('Score log_loss: ', 0.069998066356182276)


In [25]:
classes = model.predict_classes(X_test, batch_size=32)
proba = model.predict_proba(X_test, batch_size=32)
predictions_test = model.predict(X_test, batch_size=128, verbose=1)
score = log_loss(y_test, predictions_test)
print('Score log_loss: ', score)

7296/7400 [============================>.] - ETA: 0s('Score log_loss: ', 0.065347441702850631)


In [26]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

('Test score:', 0.064676598936847934)
('Test accuracy:', 0.98540540540540544)


In [28]:
y_test[1]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])

In [29]:
print log_loss(y_test[0],predictions_test[0])


nan


/home/mohammad/.local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1662: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/mohammad/.local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1662: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


In [30]:
np.round(predictions_test[0],1)

array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.], dtype=float32)